In [0]:
spark.conf.set(
"fs.azure.account.key.goodreadsreviews60300832.dfs.core.windows.net",
"a key"
)

In [0]:
books = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()

+---------+----+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|author_id|role|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+---------+----+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+-

In [0]:
"author_id" in books.columns

True

In [0]:
p = "abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/processed/reviews/"
spark.catalog.clearCache()                   # avoid stale schema
df = spark.read.parquet(p)
print(df.columns)
df.printSchema()


['user_id', 'book_id', 'review_id', 'rating', 'review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments']
root
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- review_text: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- read_at: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- n_comments: long (nullable = true)



In [0]:
from pyspark.sql.functions import col, length, trim, count, when
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/processed/reviews/"
    )
# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()
# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) == "") ).count()
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id:{null_user_id}, NULL rating: {null_rating}")

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length

# Start from the existing Parquet-loaded DataFrame
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/"))
df = reviews

# 1) Drop rows missing critical keys
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps)
df = df.dropDuplicates(["review_id"])

# 5) Select final shape
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text",
    "n_votes",
    "date_added"
)

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean.write.mode("overwrite").parquet(
"abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/processed/reviews/"
)
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+-------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------------------+
|review_id                       |book_id|user_id                         |rating|review_text                      

In [0]:
base = "abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net"

books  = spark.read.parquet(f"{base}/processed/books/")
authors = spark.read.parquet(f"{base}/processed/authors/")

reviews_clean = spark.read.parquet(f"{base}/processed/reviews/")


In [0]:
base = "abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net"
books   = spark.read.parquet(f"{base}/processed/books/")
authors = spark.read.parquet(f"{base}/processed/authors/")
reviews_clean = spark.read.parquet(f"{base}/processed/reviews/")

In [0]:
book_authors = books.select("book_id", "author_id").dropna().dropDuplicates()
book_authors.show(5)

+--------+---------+
| book_id|author_id|
+--------+---------+
| 4541271|    16667|
|35452242| 16209952|
|11890850|  4999210|
|11497621|     8715|
|12388485|  4028753|
+--------+---------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col

gold = (
    reviews_clean.alias("r")
    .join(books.alias("b"), on="book_id", how="inner")
    .join(book_authors.alias("ba"), on="book_id", how="left")
    .join(authors.alias("a"), col("ba.author_id")==col("a.author_id"), how="left")
    .select(
        col("r.review_id"),
        col("r.book_id"),
        col("b.title"),
        col("a.author_id"),
        col("a.name"),
        col("r.user_id"),
        col("r.rating"),
        col("r.review_text"),
        col("b.language_code"),      # from books
        col("r.n_votes"),       # from reviews_clean (if kept)
        col("r.date_added")     # from reviews_clean (if kept)
    )
)
gold.printSchema()
gold.show(10, truncate=False)


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+---------------------------------------------+---------+-----------------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
base = "abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net"
gold_path = f"{base}/gold/curated_reviews"

# 1) Persist Delta files for Fabric (path-based)
gold.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(gold_path)

# 2) Register a MANAGED table from the DataFrame (no path read in SQL)
spark.sql("DROP TABLE IF EXISTS hive_metastore.default.curated_reviews")
gold.write.format("delta").mode("overwrite").saveAsTable("hive_metastore.default.curated_reviews")

# 3) Verify
spark.sql("SELECT COUNT(*) FROM hive_metastore.default.curated_reviews").show()
spark.sql("DESCRIBE DETAIL hive_metastore.default.curated_reviews").show(truncate=False)


+--------+
|count(1)|
+--------+
|55009677|
+--------+

+------+------------------------------------+--------------------------------------+-----------+-----------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+-------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+-------------+
|format|id                                  |name                                  |description|location                                 |createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                           |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                                                     |clusterByAuto|
+------+------------------------------------+-------------------------------------